In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join('..')))

In [2]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

from einops import parse_shape, rearrange, repeat

def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [3]:
sequence_length = 100 # L
batch_size = 10 # B
input_size = 15 # N
hidden_size = 20 # H == output_size

rnn = nn.GRU(input_size=input_size, hidden_size=hidden_size, num_layers=1)

input = torch.randn(sequence_length, batch_size, input_size)
h0 = torch.randn(1, batch_size, hidden_size)
output, hn = rnn(input, h0)

out_shape = parse_shape(output, 'L B H')
hn_shape = parse_shape(hn, '_ B H')

print(out_shape, hn_shape)

# Good thing is that we can provide inputs of different lengths.
# Bad thing is that the entire provided sequence is computed sequentially. 

{'L': 100, 'B': 10, 'H': 20} {'B': 10, 'H': 20}


In [4]:
from brainle.models.architectures.attention import CausalSelfAttention

net = CausalSelfAttention(
    embedding_dim = 512,
    num_heads = 4, 
    block_size = 100,
    dropout_attention = 0.5,
    dropout_residual = 0.5
)

net(torch.rand(1, 100, 512)).shape
count_parameters(net)

1050624

In [13]:
from brainle.models.architectures.attention import SelfMemoryEncode, SelfMemoryDecode

encode = SelfMemoryEncode(
    embedding_dim = 1024,
    num_heads = 8,
    memory_size = 4096,
    kernel_size = 4,
    stride = 2,
    padding = 1
)   

x = torch.rand(1, 2048, 1024)
print(x.shape)

z = encode(encode(encode(x))) 
print(z.shape)

decode = SelfMemoryDecode(
    embedding_dim = 1024,
    num_heads = 8,
    memory_size = 4096,
    kernel_size = 4,
    stride = 2,
    padding = 1
)

out = decode(decode(decode(z)))
print(out.shape)

print(f"Encode params: {count_parameters(encode)}")
print(f"Decode params: {count_parameters(decode)}")

# Receptive field with L encode layers:
k = 4
L = 6
receptive_field = 2**L * (k-1) - k + 2 # ... exponential in L
print(f"""
With L={L}, kernel_size={k}, stride=2 we have a receptive field of {receptive_field} tokens at L=0, and:
* With an average english sentence of ~20 words at 5 letters per word we get max {receptive_field / 100} sentences of attention per token at L={L} 
""")

torch.Size([1, 2048, 1024])
torch.Size([1, 256, 1024])
torch.Size([1, 2048, 1024])
Encode params: 2758688
Decode params: 2758784

With L=6, kernel_size=4, stride=2 we have a receptive field of 190 tokens at L=0, and:
* With an average english sentence of ~20 words at 5 letters per word we get max 1.9 sentences of attention per token at L=6 



In [6]:
from datasets import load_dataset

dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split='train')

Reusing dataset wikitext (/Users/flavioschneider/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
